# finding motivating examples for SV

In [ ]:
import torch
import sys
import os
import numpy as np
from umap.umap_ import find_ab_params

from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.losses import SingleVisLoss, UmapLoss, ReconstructionLoss
from singleVis.trainer import SingleVisTrainer
from singleVis.data import DataProvider
from singleVis.visualizer import visualizer
import singleVis.config as config


In [ ]:
DATASET = "cifar10"
CONTENT_PATH = "/home/xianglin/projects/DVI_data/TemporalExp/resnet18_{}".format(DATASET)

In [ ]:

LEN = config.dataset_config[DATASET]["TRAINING_LEN"]
LAMBDA = config.dataset_config[DATASET]["LAMBDA"]

# define hyperparameters

DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
PATIENT = config.dataset_config[DATASET]["training_config"]["PATIENT"]
EPOCH_START = config.dataset_config[DATASET]["EPOCH_START"]
EPOCH_END = config.dataset_config[DATASET]["EPOCH_END"]
EPOCH_PERIOD = config.dataset_config[DATASET]["EPOCH_PERIOD"]

content_path = CONTENT_PATH
sys.path.append(content_path)

from Model.model import *
net = resnet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

data_provider = DataProvider(content_path, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, verbose=1)
model = SingleVisualizationModel(input_dims=512, output_dims=2, units=256)
negative_sample_rate = 5
min_dist = .1
_a, _b = find_ab_params(1.0, min_dist)
umap_loss_fn = UmapLoss(negative_sample_rate, DEVICE, _a, _b, repulsion_strength=1.0)
recon_loss_fn = ReconstructionLoss(beta=1.0)
criterion = SingleVisLoss(umap_loss_fn, recon_loss_fn, lambd=LAMBDA)

optimizer = torch.optim.Adam(model.parameters(), lr=.01, weight_decay=1e-5)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=.1)

trainer = SingleVisTrainer(model, criterion, optimizer, lr_scheduler, edge_loader=None, DEVICE=DEVICE)
trainer.load(file_path=os.path.join(data_provider.model_path,"tnn.pth"))
# trainer.load(file_path=os.path.join(data_provider.model_path,"motivated_SV.pth"))

## Find a stable sample

In [ ]:
prev_data = data_provider.train_representation(8)
curr_data = data_provider.train_representation(9)

dists = np.linalg.norm(prev_data - curr_data, axis=1)
dists.shape

In [ ]:
dists.min(), dists.mean(), dists.max(), np.argmin(dists)

In [ ]:
from singleVis.backend import find_neighbor_preserving_rate

npr = find_neighbor_preserving_rate(prev_data, curr_data, n_neighbors=15)
npr[7450]

In [ ]:
np.histogram(npr)

## Visualization

In [ ]:
vis_list = [7450]

In [ ]:
vis = visualizer(data_provider, trainer.model, 200, 10, classes)
save_dir = os.path.join(data_provider.content_path, "img")
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
for i in [1,4,10]:
    curr_data = data_provider.train_representation(i)
    pred = data_provider.get_pred(i, curr_data)
    pred = np.argmax(pred, axis=1)
    labels = data_provider.train_labels(i)
    # vis.savefig_cus(i, curr_data[vis_list], pred[vis_list], labels[vis_list], path=os.path.join(save_dir,"motivated_{}_{}.png".format(DATASET, i)))
    vis.savefig_cus(i, curr_data, pred, labels, path=os.path.join(save_dir,"motivated_{}_{}.png".format(DATASET, i)))

In [ ]:
vis = visualizer(data_provider, trainer.model, 200, 10, classes)
save_dir = os.path.join(data_provider.content_path, "img")
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
for i in range(5, 12, 1):
    curr_data = data_provider.train_representation(i)
    pred = data_provider.get_pred(i, curr_data)
    pred = np.argmax(pred, axis=1)
    labels = data_provider.train_labels(i)

    prev_data = data_provider.train_representation(i-1)
    prev_pred = data_provider.get_pred(i-1, prev_data)
    prev_pred = np.argmax(prev_pred, axis=1)
    prev_labels = data_provider.train_labels(i-1)

    vis.savefig_trajectory(i, prev_data[vis_list], prev_pred[vis_list], prev_labels[vis_list], curr_data[vis_list], pred[vis_list], labels[vis_list], path=os.path.join(save_dir,"motivated_{}_{}.png".format(DATASET, i)))

# visualize trajectory

In [ ]:
all_data = None
embeddings = torch.Tensor([]).to("cpu")
for i in range(1, TIME_STEPS+1, 1):
    data = data_provider.train_representation(i)
    if all_data is None:
        all_data = data[:, None,:]
    else:
        all_data = np.concatenate((all_data, data[:, None, :]), axis=1)
    data = torch.from_numpy(data).to(device=data_provider.DEVICE, dtype=torch.float)
    embedding = trainer.model.encoder(data).detach().cpu()
    embeddings = torch.concat((embeddings, embedding[:, None, :]), dim=1)
embeddings = embeddings.numpy()
embeddings.shape

In [ ]:
import singleVis.trajectory as trajectory
# trajectory.draw_trajectory(embeddings[7450][4:],x_min=-6.0, x_max=-2.5, y_min=5.5, y_max=7.1)
trajectory.draw_trajectory(embeddings[7450][4:], x_min=-4, x_max=-1, y_min=-4.1, y_max=-2.9)

## Ground truth of dists change and direction change

In [ ]:
all_data.shape
high_dists = np.zeros((50000, 10))
anchor = 8
for i in range(50000):
    for j in range(10):
        high_dists[i][j] = np.linalg.norm(all_data[i][anchor]-all_data[i][j])
low_dists = np.zeros((50000, 10))
for i in range(50000):
    for j in range(10):
        low_dists[i][j] = np.linalg.norm(embeddings[i][anchor]-embeddings[i][j])


In [ ]:
high_dists[101],low_dists[101]

In [ ]:
from scipy.spatial import distance
high_directions = np.zeros((50000, 9))
for i in range(50000):
    for j in range(9):
        high_directions[i][j] = 1 - distance.cosine(all_data[i][j]-all_data[i][j+1], all_data[i][j+1]-all_data[i][j+2])
low_directions = np.zeros((50000, 9))
for i in range(50000):
    for j in range(9):
        low_directions[i][j] = 1 - distance.cosine(embeddings[i][j]-embeddings[i][j+1],embeddings[i][j+1]-embeddings[i][j+2])

In [ ]:
high_directions[7450],low_directions[7450]

# correlation betweem high dim dists and low dim dists

In [ ]:
prev_data = data_provider.train_representation(9)
curr_data = data_provider.train_representation(11)

dists = np.linalg.norm(prev_data - curr_data, axis=1)

prev_data = torch.from_numpy(prev_data).to(device=data_provider.DEVICE, dtype=torch.float)
prev_embedding = trainer.model.encoder(prev_data).detach().cpu().numpy()

curr_data = torch.from_numpy(curr_data).to(device=data_provider.DEVICE, dtype=torch.float)
curr_embedding = trainer.model.encoder(curr_data).detach().cpu().numpy()

embedding_dists = np.linalg.norm(prev_embedding-curr_embedding, axis=1)

In [ ]:
from scipy.stats.stats import pearsonr
corr = pearsonr(dists, embedding_dists)
corr